# Personal Finances Summary Dashboard

This project was developed to gain a deeper understanding of my spending habits and financial trends. By leveraging my own bank statements, I used pandas and numpy for advanced data manipulation and Google Drive API for seamless data importation. I meticulously categorized and analyzed transactions, employing interactive and informative visualizations to shed light on spending patterns. This endeavor stands as a testament to the practical application of data science methodologies in fostering financial literacy and enabling more informed financial decision-making.

## Import packages


In [ ]:
#required for manipulating data
import pandas as pd
import numpy as np 

#enable Google Drive API
import gspread

#required for building the interactive dashboard
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

## Connect to the bank data using Google drive API and clean up  transactions

In [ ]:
#connect to Google Drive
#Importante note: move json file to project directory and rename it "service_account.json"

gc = gspread.service_account(filename="service_account.json")
sh = gc.open("bank of america")


ws = sh.worksheet('bank of america')
df = pd.DataFrame(ws.get_all_records())
df.head()


## Data cleaning

In [ ]:
df = df[['Date', 'Description', 'Amount']] #keep only desired columns
df['Description'] = df['Description'].map(str.lower) #lower case of descriptions
df['Category'] = 'unassigned'                        #add category column

df.head()

In [ ]:
df = df.drop(0) # Drop the first row from the DataFrame
df.head()

## Defining all categories

From my bank statetment the I defined the following categories:

* Pet
* Groceries
* Restaurants
* Entertainment
* Services
* Online Shopping
* Coffee
* Transport
* Excluded
* Selfcare
* Shopping
  

# Assign transactions to the correct category

In [ ]:


# Pet

df['Category'] = np.where(df['Description'].str.contains(
    'petsmart|vet|clt vet'), 
    'Pet', df['Category'] )
    
# Groceries

df['Category'] = np.where(df['Description'].str.contains(
    'harris teeter|costco whse|wal wal-mart|wal-mart|food lion|wm supercenter|trader joe|harris te 4701'), 
    'Groceries', df['Category'] )
    
# Restaurants

df['Category'] = np.where(df['Description'].str.contains(
    'jersey mikes|inizio|taco bell|good wurst|macalisters |jack in the box|macdonalds|rincon latino|bojangles|bp|soul foo|cook out|kfc|freddy|sonic|olive garden|chick-fil-a|lada'), 
    'Restaurants', df['Category'] )

# Entertainment
    
df['Category'] = np.where(df['Description'].str.contains(
    'prime video|nintendo|us national whitewater|national whitewa|charlotte pride '), 
    'Entertainment', df['Category'] )

# Services
    
df['Category'] = np.where(df['Description'].str.contains(
    'apple|onstar|mecklenburg ab|vzwrlss|endurance fcu|dukeenergy|at&t|geico'), 
    'Services', df['Category'] )  

# Online Shopping
    
df['Category'] = np.where(df['Description'].str.contains(
    'amzn mktp us|amazon'), 
    'Online Shopping', df['Category'] )
    

# Coffee

df['Category'] = np.where(df['Description'].str.contains(
    'starbucks'), 
    'Coffee', df['Category'] )

# Transport
    
df['Category'] = np.where(df['Description'].str.contains(
    'uber|shell service|shell oil'), 
    'Transport', df['Category'] )

# Excluded
    
df['Category'] = np.where(df['Description'].str.contains(
    'keep the change|banking transfer|corning incorpor|corning|starnow|refund|giffgaff|backstage|hectagon|tower hamlets bc|sweet suites|temporary hold|cm.com'), 
    'Excluded', df['Category'] )

# Selfcare
    
df['Category'] = np.where(df['Description'].str.contains(
    'the cut barbershop'), 
    'Selfcare', df['Category'] )

# Shopping
    
df['Category'] = np.where(df['Description'].str.contains(
    'target|kohls|sp new york filthy'), 
    'Shopping', df['Category'] )
    


In [ ]:
# Convert the "Date" column to a datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract the month and year information
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
    
pd.options.display.max_rows = 999
df.head(200)

In [ ]:
# Extract the month and year information
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
    
pd.options.display.max_rows = 999
df.head(200)

In [ ]:
#check unassigned transactions and confirm all transactions are assigned to a category

unassigned = df.loc[df['Category'] == 'unassigned']
unassigned

## Create Top Banner for a summary of last month's income, recurring expenses, non-recurring expenses and savings

In [ ]:
# Get the latest month and year
latest_month = df['Month'].max()
latest_year = df['Year'].max()

In [ ]:
#Filter the dataframe to include only transactions from the latest month
last_month_expenses = df[(df['Month'] == latest_month) & (df['Year'] == latest_year)]


In [ ]:
last_month_expenses = last_month_expenses.groupby('Category')['Amount'].sum().reset_index()

last_month_expenses['Amount']=last_month_expenses['Amount'].astype('str')
last_month_expenses['Amount']=last_month_expenses['Amount'].str.replace('-','')
last_month_expenses['Amount']=last_month_expenses['Amount'].astype('float')        #get absolute figures

last_month_expenses = last_month_expenses[last_month_expenses["Category"].str.contains("Excluded|unassigned") == False]    #exclude "excluded" category
last_month_expenses = last_month_expenses.sort_values(by='Amount', ascending=False)    #sort values
last_month_expenses['Amount'] = last_month_expenses['Amount'].round().astype(int)      #round values

last_month_expenses

In [ ]:
last_month_expenses_tot = last_month_expenses['Amount'].sum()
last_month_expenses_tot



In [ ]:
def calculate_difference(event):
    income = float(income_widget.value)
    recurring_expenses = float(recurring_expenses_widget.value)
    monthly_expenses = float(monthly_expenses_widget.value)
    difference = income - recurring_expenses - monthly_expenses
    difference_widget.value = str(difference)

income_widget = pn.widgets.TextInput(name="Income", value="0")
recurring_expenses_widget = pn.widgets.TextInput(name="Recurring Expenses", value="0")
monthly_expenses_widget = pn.widgets.TextInput(name="Non-Recurring Expenses", value=str(last_month_expenses_tot))
difference_widget = pn.widgets.TextInput(name="Last Month's Savings", value="0")

income_widget.param.watch(calculate_difference, "value")
recurring_expenses_widget.param.watch(calculate_difference, "value")
monthly_expenses_widget.param.watch(calculate_difference, "value")

#pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget).show()


## Creating last month expenses bar chart 

In [ ]:
last_month_expenses_chart = last_month_expenses.hvplot.bar(
    x='Category', 
    y='Amount', 
    height=250, 
    width=850, 
    title="Last Month Expenses",
    ylim=(0, 500))

last_month_expenses_chart


## Creating monthly expenses trend bar chart 

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])            # convert the 'Date' column to a datetime object
df['Month-Year'] = df['Date'].dt.to_period('M')    # extract the month and year from the 'Date' column and create a new column 'Month-Year'
monthly_expenses_trend_by_cat = df.groupby(['Month-Year', 'Category'])['Amount'].sum().reset_index()

monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].astype('str')
monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].str.replace('-','')
monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].astype('float')
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat["Category"].str.contains("Excluded") == False]

monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.sort_values(by='Amount', ascending=False)
monthly_expenses_trend_by_cat['Amount'] = monthly_expenses_trend_by_cat['Amount'].round().astype(int)
monthly_expenses_trend_by_cat['Month-Year'] = monthly_expenses_trend_by_cat['Month-Year'].astype(str)
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.rename(columns={'Amount': 'Amount '})

monthly_expenses_trend_by_cat

## Defining Panel widget

In [ ]:
select_category1 = pn.widgets.Select(name='Select Category', options=[
    'All',
    'Pet',
    'Groceries',
    'Restaurants',
    'Entertainment',
    'Services',
    'Online Shopping',
    'Coffee',
    'Transport',
    'Excluded',
    'Selfcare',
    'Shopping',
    #'Excluded'
   
])

select_category1


In [ ]:
# define plot function
def plot_expenses(category):
    if category == 'All':
        plot_df = monthly_expenses_trend_by_cat.groupby('Month-Year').sum()
    else:
        plot_df = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat['Category'] == category].groupby('Month-Year').sum()
    plot = plot_df.hvplot.bar(x='Month-Year', y='Amount ')
    return plot

# define callback function
@pn.depends(select_category1.param.value)
def update_plot(category):
    plot = plot_expenses(category)
    return plot

# create layout
monthly_expenses_trend_by_cat_chart = pn.Row(select_category1, update_plot)
monthly_expenses_trend_by_cat_chart[1].width = 600

monthly_expenses_trend_by_cat_chart

## Creating summary table

In [ ]:
df = df[['Date', 'Category', 'Description', 'Amount']]
df['Amount']=df['Amount'].astype('str')
df['Amount']=df['Amount'].str.replace('-','')
df['Amount']=df['Amount'].astype('float')        #get absolute figures

df = df[df["Category"].str.contains("Excluded") == False]    #exclude "excluded" category
df['Amount'] = df['Amount'].round().astype(int)      #round values
df



In [ ]:
# Define a function to filter the dataframe based on the selected category
def filter_df(category):
    if category == 'All':
        return df
    return df[df['Category'] == category]


In [ ]:
# Create a DataFrame widget that updates based on the category filter
summary_table = pn.widgets.DataFrame(filter_df('All'), height = 300,width=400)

# Define a callback that updates the dataframe widget when the category filter is changed
def update_summary_table(event):
    summary_table.value = filter_df(event.new)

# Add the callback function to the category widget
select_category1.param.watch(update_summary_table, 'value')

summary_table

## Create Final Dashboard

In [ ]:
template = pn.template.FastListTemplate(
    title="Personal Finances Summary",
    sidebar=[
        pn.pane.Markdown("## *If you can't manage your money, making more won't help*"),
        pn.pane.JPG('bill.jpg', sizing_mode='scale_both'),
        pn.pane.Markdown(""),
        pn.pane.Markdown(""),
        select_category1
    ],
    main=[
        pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget, width=950),
        pn.Row(last_month_expenses_chart, height=240),
        pn.GridBox(
            monthly_expenses_trend_by_cat_chart[1],
            summary_table,
            ncols=2,
            width=500,  
            align='start',
            sizing_mode='stretch_width'
        )
    ]
)

template.show()